In [8]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

from dotenv import load_dotenv
import os

# Încarcă variabilele de mediu din fișierul .env
load_dotenv()
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

def returnAllWordsThatAppearInImage(imagePath):
    """ 
    Returns a list of words that are detected in an image given by its path
    """
    img = open(imagePath, "rb")
    read_response = computervision_client.read_in_stream(
        image=img,
        mode="Printed",
        raw=True
    )
    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    
    words = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                for word in line.text.split():
                    words.append(word)
    return words


def returnAllCharactersThatAppearInAListOfWords(words):
    """  
    Returns all characters that appear in a list of words
    words: list of words
    """
    characters = []
    for word in words:
        for character in word:
            characters.append(character)
    return characters

In [22]:
def distantaLevenshtein(str1, str2):
    # str1 is empty
    if len(str1) == 0:
        return len(str2)
    # str2 is empty
    if len(str2) == 0:
        return len(str1)
    if str1[-1] == str2[-1]:
        return distantaLevenshtein(str1[0:len(str1)-1], str2[0:len(str2)-1])
    return 1 + min(
          # Insert     
        distantaLevenshtein(str1, str2[0:len(str2)-1]),
        min(
              # Remove
            distantaLevenshtein(str1[0:len(str1)-1], str2),
          # Replace
            distantaLevenshtein(str1[0:len(str1)-1], str2[0:len(str2)-1])
        )    
    )

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Levenshtein
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = distantaLevenshtein(recognisedWords, normalWords)
    calitateaCaracterelor = distantaLevenshtein(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor


def hamming_distance(string1: str, string2: str) -> int:
    """Return the Hamming distance between two strings."""
    if len(string1) != len(string2):
        raise ValueError("Strings must be of equal length.")
    dist_counter = 0
    for n in range(len(string1)):
        if string1[n] != string2[n]:
            dist_counter += 1
    return dist_counter

def calitateaProcesuluiDeRecunoastereATextuluiCuDistantaHamming(imagePath, normalWords, normalCharacters):
    """ 
    Masoara calitatea procesului de recunoastere a cuvintelor, respectiv caracterelor folosind distanta Hamming
    normalWords - cuvintele din imagine
    normalCharacters - literele din imagine
    """
    recognisedWords = returnAllWordsThatAppearInImage(imagePath)
    recognisedCharacters = returnAllCharactersThatAppearInAListOfWords(recognisedWords)

    calitateaCuvintelor = hamming_distance(recognisedWords, normalWords)
    calitateaCaracterelor = hamming_distance(recognisedCharacters, normalCharacters)

    return calitateaCuvintelor, calitateaCaracterelor

In [23]:
def main():
    # words = ["Succes", "în", "rezolvarea", "tEMELOR", "la", "LABORAtoarele", "de", "Inteligență", "Artificală!"]
    # results = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein("test2.jpeg", words, returnAllCharactersThatAppearInAListOfWords(words))
    # print("Cuvinte corecte folosind distanta Levenshtein = ", results[0])
    # print("Caractere corecte folosind distanta Levenshtein = ", results[1])

    imagePath = "test1.png"
    words = ["Google", "Cloud", "Platform"]
    characters = returnAllCharactersThatAppearInAListOfWords(words)
    resultsLevenshtein = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaLevenshtein(imagePath, words,characters)
    print("Cuvinte corecte folosind distanta Levenshtein: {} din {}".format( len(words) - resultsLevenshtein[0], len(words)))
    print("Caractere corecte folosind distanta Levenshtein: {} din {}".format( len(characters) - resultsLevenshtein[1], len(characters)))

    resultsHamming = calitateaProcesuluiDeRecunoastereATextuluiCuDistantaHamming(imagePath, words, characters)
    print("Cuvinte corecte folosind distanta Hamming: {} din {}".format( len(words) - resultsHamming[0], len(words)))
    print("Caractere corecte folosind distanta Hamming: {} din {}".format( len(characters) - resultsHamming[1], len(characters)))

    

main()

Cuvinte corecte folosind distanta Levenshtein: 3 din 3
Caractere corecte folosind distanta Levenshtein: 19 din 19
Cuvinte corecte folosind distanta Hamming: 3 din 3
Caractere corecte folosind distanta Hamming: 19 din 19
